In [2]:
import os
import numpy as np
from PIL import Image
from Anisotropic import *
import random
import tensorflow as tf
from tensorflow.keras import layers, models

def extract_random_windows(original_img, filtered_img, k, N):
    """
    Extract random k x k windows from both original and filtered images.
    
    Args:
    - original_img (np.array): The original grayscale image.
    - filtered_img (np.array): The anisotropic filtered grayscale image.
    - k (int): The size of the window (k x k).
    - N (int): The number of windows to extract.
    
    Returns:
    - windows (list of tuples): A list of (xi, yi) window pairs.
    """
    windows = []
    img_height, img_width = original_img.shape
    
    for _ in range(N):
        # Randomly select top-left corner of the window
        x = random.randint(0, img_width - k)
        y = random.randint(0, img_height - k)
        
        # Extract windows
        xi = original_img[y:y+k, x:x+k]
        yi = filtered_img[y:y+k, x:x+k]
        
        windows.append((xi, yi))
    
    return windows

def create_dataset(input_folder, output_folder, k, N, train_ratio=0.7, val_ratio=0.15):
    """
    Create a dataset of (xi, yi) pairs from images in input_folder.
    
    Args:
    - input_folder (str): Path to the folder containing original images.
    - output_folder (str): Path to the folder containing filtered images.
    - k (int): The size of the window (k x k).
    - N (int): The total number of windows to extract.
    - train_ratio (float): Proportion of the dataset for training.
    - val_ratio (float): Proportion of the dataset for validation.
    
    Returns:
    - train_set (list): List of (xi, yi) pairs for training.
    - val_set (list): List of (xi, yi) pairs for validation.
    - test_set (list): List of (xi, yi) pairs for testing.
    """
    windows = []

    # Iterate through all files in the input folder
    for filename in os.listdir(input_folder):
        # Check if the file is an image
        if filename.endswith(".jpg") or filename.endswith(".png"):
            # Load original and filtered images
            original_img_path = os.path.join(input_folder, filename)
            filtered_img_path = os.path.join(output_folder, filename)
            
            original_img = np.array(Image.open(original_img_path).convert('L'))
            filtered_img = np.array(Image.open(filtered_img_path))
            
            # Extract windows
            windows.extend(extract_random_windows(original_img, filtered_img, k, N // len(os.listdir(input_folder))))

    # Shuffle the dataset
    random.shuffle(windows)

    # Split the dataset
    train_size = int(len(windows) * train_ratio)
    val_size = int(len(windows) * val_ratio)
    
    train_set = windows[:train_size]
    val_set = windows[train_size:train_size + val_size]
    test_set = windows[train_size + val_size:]
    
    return train_set, val_set, test_set

# Parameters
input_folder_path = "images"
output_folder_path = "bw-images"
k = 32  # Window size
N = 500000  # Number of windows to extract

# Create dataset
train_set, val_set, test_set = create_dataset(input_folder_path, output_folder_path, k, N)

# Save or use the datasets as needed
# For example, you might want to save them to disk or use them directly for training a model.


In [ ]:
def unet_model(input_size=(32, 32, 1)):
    inputs = layers.Input(input_size)

    # Down-sampling path
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    # Bottleneck
    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c4)

    # Up-sampling path
    u5 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c4)
    u5 = layers.concatenate([u5, c3])
    c5 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(u5)
    c5 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c5)

    u6 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = layers.concatenate([u6, c2])
    c6 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u6)
    c6 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c6)

    u7 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = layers.concatenate([u7, c1])
    c7 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u7)
    c7 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c7)

    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(c7)

    model = models.Model(inputs=[inputs], outputs=[outputs])
    return model

In [ ]:
def prepare_data(data_set, k):
    """
    Prepares data for training.
    
    Args:
    - data_set (list of tuples): List of (xi, yi) pairs.
    - k (int): Size of the windows.
    
    Returns:
    - X (np.array): Input data of shape (?, k, k, 1).
    - y (np.array): Target data of shape (?, k, k, 1).
    """
    X = np.array([xi.reshape((k, k, 1)) for xi, _ in data_set])
    y = np.array([yi.reshape((k, k, 1)) for _, yi in data_set])
    return X, y

k = 32  # Window size
train_X, train_y = prepare_data(train_set, k)
val_X, val_y = prepare_data(val_set, k)
test_X, test_y = prepare_data(test_set, k)


In [ ]:
# Define the model
model = unet_model(input_size=(k, k, 1))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_X, train_y, epochs=20, batch_size=32, validation_data=(val_X, val_y))


In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_X, test_y)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_acc}')


In [3]:
import os
import numpy as np
from PIL import Image
from Anisotropic import *
import random
import tensorflow as tf
from tensorflow.keras import layers, models

def extract_random_windows(original_img, filtered_img, k, N):
    windows = []
    img_height, img_width = original_img.shape
    
    for _ in range(N):
        x = random.randint(0, img_width - k)
        y = random.randint(0, img_height - k)
        
        xi = original_img[y:y+k, x:x+k]
        yi = filtered_img[y:y+k, x:x+k]
        
        windows.append((xi, yi))
    
    return windows

def create_dataset(input_folder, output_folder, k, N, train_ratio=0.7, val_ratio=0.15):
    windows = []

    for filename in os.listdir(input_folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            original_img_path = os.path.join(input_folder, filename)
            filtered_img_path = os.path.join(output_folder, filename)
            
            original_img = np.array(Image.open(original_img_path).convert('L'))
            filtered_img = np.array(Image.open(filtered_img_path))
            
            windows.extend(extract_random_windows(original_img, filtered_img, k, N // len(os.listdir(input_folder))))

    random.shuffle(windows)

    train_size = int(len(windows) * train_ratio)
    val_size = int(len(windows) * val_ratio)
    
    train_set = windows[:train_size]
    val_set = windows[train_size:train_size + val_size]
    test_set = windows[train_size + val_size:]
    
    return train_set, val_set, test_set

def prepare_data(data_set, k):
    X = np.array([xi.reshape((k, k, 1)) for xi, _ in data_set])
    y = np.array([yi.reshape((k, k, 1)) for _, yi in data_set])
    return X, y

def unet_model(input_size=(32, 32, 1)):
    inputs = layers.Input(input_size)

    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c4)

    u5 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c4)
    u5 = layers.concatenate([u5, c3])
    c5 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(u5)
    c5 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c5)

    u6 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = layers.concatenate([u6, c2])
    c6 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u6)
    c6 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c6)

    u7 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = layers.concatenate([u7, c1])
    c7 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u7)
    c7 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c7)

    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(c7)

    model = models.Model(inputs=[inputs], outputs=[outputs])
    return model

# Parameters
input_folder_path = "images"
output_folder_path = "bw-images"
k = 32
N = 500000

# Create dataset
train_set, val_set, test_set = create_dataset(input_folder_path, output_folder_path, k, N)

# Prepare data
train_X, train_y = prepare_data(train_set, k)
val_X, val_y = prepare_data(val_set, k)
test_X, test_y = prepare_data(test_set, k)

# Define the model
model = unet_model(input_size=(k, k, 1))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Increase batch size to reduce the number of steps per epoch
batch_size = 512

# Train the model
history = model.fit(train_X, train_y, epochs=20, batch_size=batch_size, validation_data=(val_X, val_y))

# Evaluate the model
test_loss, test_acc = model.evaluate(test_X, test_y)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_acc}')


Epoch 1/20
250/681 ━━━━━━━━━━━━━━━━━━━━ 1:25:53 12s/step - accuracy: 3.3113e-04 - loss: nan

KeyboardInterrupt: 